# INTERSECTON

In [1]:
def mv_weights(Theta_hat, mu, target_return=0.01):
    """
    Compute Mean-Variance portfolio weights with target return.
    
    Solves the constrained optimization:
    min w' Sigma w  subject to  w' mu = target_return  and  w' 1 = 1
    
    Solution uses Lagrange multipliers with two constraints.
    
    Parameters:
    -----------
    Theta_hat : np.ndarray, shape (p, p)
        Precision matrix (Sigma^{-1})
    mu : np.ndarray, shape (p,)
        Expected returns
    target_return : float
        Target portfolio return (default: 0.01 = 1% monthly)
    long_only : bool
        If True, falls back to GMV if MV produces negative weights
    
    Returns:
    --------
    w_star : np.ndarray, shape (p,)
        Portfolio weights
    """
    p = Theta_hat.shape[0]
    ones_p = np.ones(p)
    
    # Compute key quantities
    A = ones_p @ Theta_hat @ ones_p  # 1' Theta 1
    B = ones_p @ Theta_hat @ mu       # 1' Theta mu  
    C = mu @ Theta_hat @ mu           # mu' Theta mu
    D = A * C - B * B                  # Determinant
    
    # Check for singularity
    if np.abs(D) < 1e-10:
        print('SINGULARITY')
        # System is singular, use GMV instead
        if np.abs(A) > 1e-10:
            w_star = (Theta_hat @ ones_p) / A
            return w_star
        else:
            return ones_p / p
    
    
    # Compute Lagrange multipliers
    lambda1 = (C - B * target_return) / D
    lambda2 = (A * target_return - B) / D
    
    # Compute weights: w = lambda1 * Theta^{-1} 1 + lambda2 * Theta^{-1} mu
    w_star = lambda1 * (Theta_hat @ ones_p) + lambda2 * (Theta_hat @ mu)
    
    return w_star

def msr_weights(Theta_hat, mu):
    """
    Compute Maximum Sharpe Ratio portfolio weights.
    
    The maximum Sharpe ratio portfolio solves:
    max (w' mu) / sqrt(w' Sigma w)
    
    Solution (when mu represents excess returns):
    w ∝ Sigma^{-1} mu = Theta mu
    
    Then normalize so that sum(w) = 1.
    
    Parameters:
    -----------
    Theta_hat : np.ndarray, shape (p, p)
        Precision matrix (Sigma^{-1})
    mu : np.ndarray, shape (p,)
        Expected excess returns
    
    Returns:
    --------
    w_star : np.ndarray, shape (p,)
        Portfolio weights (sum to 1)
    """
    p = Theta_hat.shape[0]
    ones_p = np.ones(p)
    
    # Compute unnormalized weights: w ∝ Theta mu
    w_unnorm = Theta_hat @ mu
    
    # Normalize to sum to 1
    weight_sum = np.sum(w_unnorm)
    
    if np.abs(weight_sum) < 1e-10:
        print('WARNING: Weight sum near zero, returning equal weights')
        return ones_p / p
    
    w_star = w_unnorm / weight_sum
    
    return w_star

In [2]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Lasso, LassoCV, LinearRegression
import warnings
warnings.filterwarnings('ignore')


def est_ndwcov_factor(Y, factors, ic, lambda_min=True):
    """
    Estimate nodewise covariance with factor models using LASSO.
    
    Parameters:
    -----------
    Y : numpy.ndarray
        n x p matrix of observations
    factors : numpy.ndarray
        n x k matrix of factors
    ic : str
        Information criterion: 'WIC', 'BIC', 'GIC', 'AIC', or 'cv'
    lambda_min : bool
        If True and ic='cv', use lambda.min; otherwise use lambda.1se
        
    Returns:
    --------
    TAU : numpy.ndarray
        p x p precision matrix estimate
    """
    # Initialization
    p = Y.shape[1]
    n = Y.shape[0]
    C = np.zeros((p, p))
    np.fill_diagonal(C, 1)
    tau = []
    ns1 = np.ones((n, 1))
    
    # Fit factor model: Y = factors * beta + u
    # Add intercept to factors
    factors_with_intercept = np.column_stack([np.ones(n), factors])
    
    # Fit linear regression for each column of Y
    factormodel = LinearRegression(fit_intercept=False)
    factormodel.fit(factors_with_intercept, Y)
    
    # Get residuals and beta coefficients (excluding intercept)
    u = Y - factormodel.predict(factors_with_intercept)
    beta = factormodel.coef_[:, 1:]  # p x k matrix (excluding intercept)
    
    # Loop over the assets
    for j in range(p):
        # Create design matrix excluding column j
        X_j = np.delete(u, j, axis=1)
        y_j = u[:, j]
        
        if ic != 'cv':
            # Fit LASSO path
            alphas = np.logspace(-4, 1, 100)  # Create lambda sequence
            df_list = []
            sig_list = []
            bic_list = []
            coef_list = []
            res_list = []
            
            for alpha in alphas:
                model = Lasso(alpha=alpha, fit_intercept=False, max_iter=10000)
                model.fit(X_j, y_j)
                
                # Predictions and residuals
                y_pred = model.predict(X_j)
                res = y_j - y_pred
                
                # Degrees of freedom (number of non-zero coefficients)
                df = np.sum(np.abs(model.coef_) > 1e-8)
                
                # Variance of residuals
                sig = np.sum(res**2) / n
                
                # Compute information criterion
                if ic == 'WIC':
                    bic_val = np.log(sig) + df * np.log(n) / n * np.log(np.log(p))
                elif ic == 'BIC':
                    bic_val = np.log(sig) + df * np.log(n) / n
                elif ic == 'GIC':
                    bic_val = np.log(sig) + df * np.log(p) * np.log(np.log(n)) / n
                elif ic == 'AIC':
                    bic_val = np.log(sig) + 2 * df
                else:
                    raise ValueError(f"Unknown IC: {ic}")
                
                df_list.append(df)
                sig_list.append(sig)
                bic_list.append(bic_val)
                coef_list.append(model.coef_.copy())
                res_list.append(res)
            
            # Select model with minimum IC
            jind = np.argmin(bic_list)
            jpar = coef_list[jind]
            jres = res_list[jind]
            jtau = np.sum(y_j * jres) / n
            
        else:  # Cross-validation
            lasso_cv = LassoCV(cv=5, fit_intercept=False, max_iter=10000, n_alphas=100)
            lasso_cv.fit(X_j, y_j)
            
            if lambda_min:
                # Use alpha that minimizes CV error (lambda.min equivalent)
                jfit = lasso_cv.predict(X_j)
                jpar = lasso_cv.coef_
            else:
                # Use alpha within 1 SE of minimum (lambda.1se equivalent)
                cv_scores = lasso_cv.mse_path_.mean(axis=1)
                cv_std = lasso_cv.mse_path_.std(axis=1)
                min_idx = np.argmin(cv_scores)
                threshold = cv_scores[min_idx] + cv_std[min_idx]
                
                # Find largest alpha with CV score below threshold
                valid_indices = np.where(cv_scores <= threshold)[0]
                se_idx = valid_indices[0] if len(valid_indices) > 0 else min_idx
                
                selected_alpha = lasso_cv.alphas_[se_idx]
                model_1se = Lasso(alpha=selected_alpha, fit_intercept=False, max_iter=10000)
                model_1se.fit(X_j, y_j)
                jfit = model_1se.predict(X_j)
                jpar = model_1se.coef_
            
            jres = y_j - jfit
            jtau = np.sum(y_j * jres) / n
        
        # Fill in C matrix
        # Insert coefficients back (accounting for missing j-th position)
        C_row = np.insert(-jpar / jtau, j, 0)
        C[j, :] = C_row
        tau.append(jtau)
    
    # Set diagonal
    np.fill_diagonal(C, 1 / np.array(tau))
    omega = C.copy()
    omegasym = (C + C.T) / 2
    
    # Compute factor covariance - ensure float64
    covft = (1/n) * (factors.T @ factors) - (1/(n**2)) * (factors.T @ ns1 @ ns1.T @ factors)
    covft = covft.astype(np.float64)
    
    # Ensure beta and omegasym are float64
    beta = beta.astype(np.float64)
    omegasym = omegasym.astype(np.float64)
    

    covft_inv = np.linalg.inv(covft)
    p1 = np.linalg.inv(covft_inv + beta.T @ omegasym @ beta)
    TAU = omega - omega @ beta @ p1 @ beta.T @ omega
    
    return TAU


def gmv_weights(Theta_hat):
    """
    Compute Global Minimum Variance (GMV) portfolio weights (Section 6.1).
    
    Parameters:
    -----------
    Theta_hat : np.ndarray, shape (p, p)
        Precision matrix
    
    Returns:
    --------
    w_star : np.ndarray, shape (p,)
        Portfolio weights
    """
    p = Theta_hat.shape[0]
    ones_p = np.ones(p)
    
    # w* = (Θ 1_p) / (1_p' Θ 1_p)
    numerator = Theta_hat @ ones_p
    denominator = ones_p @ Theta_hat @ ones_p
    
    if np.abs(denominator) < 1e-10:
        # Fallback to equal weights if precision matrix is near-singular
        return ones_p / p
    
    w_star = numerator / denominator
    
    return w_star


def load_yearly_signals(year, buys_path_template='buys_{}.csv', sells_path_template='sells_{}.csv'):
    """
    Load buy and sell signals for a specific year.
    
    Parameters:
    -----------
    year : int
        Year to load signals for
    buys_path_template : str
        Template for buys file path (use {} for year placeholder)
    sells_path_template : str
        Template for sells file path (use {} for year placeholder)
    
    Returns:
    --------
    permno_set : set
        Set of permnos in the buy and sell signals for this year
    """
    try:
        buys = pd.read_csv(buys_path_template.format(year), index_col=1)
        sells = pd.read_csv(sells_path_template.format(year), index_col=1)
        
        buys.index.name = 'permno'
        sells.index.name = 'permno'
        
        buys_index = buys.index.astype(int)
        sells_index = sells.index.astype(int)
        
        return set(buys_index.union(sells_index))
    except FileNotFoundError as e:
        print(f"  ⚠ Warning: Could not load signals for year {year}: {e}")
        return set()

def load_ff_factors(factors_path='factors_ff_monthly_raw.csv'):
    """
    Load Fama-French factors from CSV file.
    
    Parameters:
    -----------
    factors_path : str
        Path to the factors CSV file
    
    Returns:
    --------
    factors_df : pd.DataFrame
        DataFrame with date index and factor columns
    """
    factors_df = pd.read_csv(factors_path)
    
    # Convert month column (e.g., 192707) to datetime
    # This gives us the first day of the month (1927-07-01)
    factors_df['date'] = pd.to_datetime(factors_df.iloc[:, 0].astype(str), format='%Y%m')
    
    # Convert to end of month to match returns data
    factors_df['date'] = factors_df['date'] + pd.offsets.MonthEnd(0)
    
    # Set date as index and keep only factor columns
    factors_df = factors_df.set_index('date')[['Mkt-RF', 'SMB', 'HML']]
    
    # Convert to decimal form (assuming factors are in percentage points)
    factors_df = factors_df / 100
    
    return factors_df

def load_analyst_recommendations(rec_changes_path):
    """
    Load significant recommendation changes from CSV file.
    
    Parameters:
    -----------
    rec_changes_path : str
        Path to significant_recommendation_changes.csv file
    
    Returns:
    --------
    rec_changes_df : pd.DataFrame
        DataFrame with columns: permno, date, ticker, mean_recommendation, 
        recommendation_change, num_recommendations
    """
    try:
        rec_changes_df = pd.read_csv(rec_changes_path)
        rec_changes_df['date'] = pd.to_datetime(rec_changes_df['date'])
        rec_changes_df['permno'] = rec_changes_df['permno'].astype(int)
        return rec_changes_df
    except FileNotFoundError as e:
        print(f"  ⚠ Warning: Could not load recommendation changes: {e}")
        return pd.DataFrame(columns=['permno', 'date', 'ticker', 'mean_recommendation', 
                                    'recommendation_change', 'num_recommendations'])


def get_signal_permnos_for_date(rec_changes_df, date, buy_threshold=-0.5, sell_threshold=0.5):
    """
    Get sets of permnos with buy/sell signals based on recommendation changes.
    
    Note: Negative change = upgrade (moving toward Strong Buy) = BUY signal
          Positive change = downgrade (moving toward Sell) = SELL signal
    
    Parameters:
    -----------
    rec_changes_df : pd.DataFrame
        Recommendation changes dataframe
    date : pd.Timestamp
        Date to get signals for
    buy_threshold : float
        Threshold for buy signals (default: -0.5)
    sell_threshold : float
        Threshold for sell signals (default: 0.5)
    
    Returns:
    --------
    buy_permnos : set
        Set of permnos with buy signals
    sell_permnos : set
        Set of permnos with sell signals
    """
    date_changes = rec_changes_df[rec_changes_df['date'] == date]
    
    # Buy signals: negative changes (recommendations getting better)
    buys = date_changes[date_changes['recommendation_change'] <= buy_threshold]
    buy_permnos = set(buys['permno'].values)
    
    # Sell signals: positive changes (recommendations getting worse)
    sells = date_changes[date_changes['recommendation_change'] >= sell_threshold]
    sell_permnos = set(sells['permno'].values)
    
    return buy_permnos | sell_permnos


def calculate_exit_transaction_cost(prev_weights_dict, prev_oos_returns_dict, 
                                    prev_gross_return, transaction_cost, verbose=False):
    """
    Calculate transaction cost when exiting the market (liquidating all positions).
    Enforces Immediate Liquidation logic:
    - Next period return is 0.0 (Cash)
    - Cost is paid on current portfolio value
    """
    if len(prev_weights_dict) == 0:
        return 0.0, 0.0, 0.0
    
    # Step 1: Adjust previous weights to current period's BEGINNING (drift from t-1 to t)
    adjusted_prev = {}
    for asset, prev_w in prev_weights_dict.items():
        if asset in prev_oos_returns_dict:
            prev_r = prev_oos_returns_dict[asset]
            if abs(1 + prev_gross_return) > 1e-6:
                adjusted_prev[asset] = prev_w * (1 + prev_r) / (1 + prev_gross_return)
            else:
                adjusted_prev[asset] = 0.0
        else:
            if abs(1 + prev_gross_return) > 1e-6:
                adjusted_prev[asset] = prev_w / (1 + prev_gross_return)
            else:
                adjusted_prev[asset] = 0.0
    
    # Step 2: Turnover (Selling everything to Cash)
    turnover = sum(abs(w) for w in adjusted_prev.values())
    
    # Step 3: Cost 
    # Paper Formula: c * (1 + R_next) * Turnover. 
    # Since R_next (Cash) is 0.0, this simplifies to c * 1.0 * Turnover.
    tc = transaction_cost * 1.0 * turnover
    
    # Step 4: Net Return is 0.0 (Cash return) - Cost
    net_return = -tc
    
    if verbose:
        print(f"  Liquidating positions | Turnover: {turnover:>6.4f} | TC: {tc:>8.6f}")
    
    return turnover, tc, net_return


def backtest_nodewise_gmv_yearly(df, 
                                 factors_path='factors_ff_monthly_raw.csv',
                                ic='GIC',
                                  test_start_date='2020-01-31', 
                                  test_end_date='2024-11-30',
                                  lookback_window=180,
                                  transaction_cost=0.001,
                                  buys_path_template='buys_{}.csv',
                                  sells_path_template='sells_{}.csv',
                                  analyst_rec_path=None,
                                  mv_target_return=0.01,
                                  verbose=True):
    """
    Backtest Nodewise + GMV strategy with year-specific buy/sell signals 
    and analyst recommendations.
    
    Parameters:
    -----------
    df : pd.DataFrame
        DataFrame with columns: permno, datadate, ret_fwd_1
    test_start_date : str
        First date for out-of-sample returns (format: 'YYYY-MM-DD')
    test_end_date : str
        Last date for out-of-sample returns (format: 'YYYY-MM-DD')
    lookback_window : int
        Number of months in rolling training window (default: 180)
    transaction_cost : float
        Proportional transaction cost (default: 0.001 = 10 bps)
    buys_path_template : str
        Template for buys file path (use {} for year placeholder)
    sells_path_template : str
        Template for sells file path (use {} for year placeholder)
    analyst_rec_path : str or None
        Path to analyst recommendations CSV file. If None, only uses buy/sell signals.
    verbose : bool
        If True, prints detailed log at each time step.
    
    Returns:
    --------
    results_df : pd.DataFrame
        DataFrame with columns: date, portfolio_return, cumulative_return
    metrics : dict
        Overall performance metrics
    """
    # --- 1. Setup ---
    df = df.copy()
    if 'datadate' not in df.columns or 'permno' not in df.columns:
        raise ValueError("DataFrame must have 'datadate' and 'permno' columns")
    df['datadate'] = pd.to_datetime(df['datadate'])

    # Load Fama-French factors
    factors_df = load_ff_factors(factors_path)
    
    # Load analyst recommendations if provided
    analyst_df = None
    if analyst_rec_path is not None:
        analyst_df = load_analyst_recommendations(analyst_rec_path)
        if verbose and len(analyst_df) > 0:
            print(f"Loaded analyst recommendations: {len(analyst_df)} records")
    
    # Get unique dates
    all_dates = sorted(df['datadate'].unique())
    
    # Convert test dates to datetime
    test_start_dt = pd.to_datetime(test_start_date)
    test_end_dt = pd.to_datetime(test_end_date)
    
    # Find date indices
    try:
        test_start_idx = all_dates.index(test_start_dt)
        test_end_idx = all_dates.index(test_end_dt)
    except ValueError as e:
        raise ValueError(f"Date not found in DataFrame: {e}")
    
    if test_start_idx < lookback_window:
        raise ValueError(f"Not enough data for lookback. Test start date {test_start_date} "
                         f"requires data back to {all_dates[test_start_idx - lookback_window]}, "
                         f"but only {test_start_idx} periods are available.")
    
    # Storage for results - separate for each portfolio type
    portfolio_results = {
        'gmv': {'returns': [], 'dates': [], 'weights': [], 'turnover': [], 'gross_returns': []},
        'mv': {'returns': [], 'dates': [], 'weights': [], 'turnover': [], 'gross_returns': []},
        'msr': {'returns': [], 'dates': [], 'weights': [], 'turnover': [], 'gross_returns': []}
    }
    
    # Track previous state for each portfolio
    prev_state = {
        'gmv': {'weights_dict': {}, 'oos_returns_dict': {}, 'gross_return': 0.0},
        'mv': {'weights_dict': {}, 'oos_returns_dict': {}, 'gross_return': 0.0},
        'msr': {'weights_dict': {}, 'oos_returns_dict': {}, 'gross_return': 0.0}
    }
    
    # Cache for yearly signals
    yearly_signals_cache = {}
    
    # --- 2. Rolling Window Backtest ---
    if verbose:
        print("="*60)
        print("STARTING BACKTEST WITH YEARLY SIGNALS + ANALYST RECS")
        print("="*60)
        
    for t in range(test_start_idx, test_end_idx + 1):
        current_date = all_dates[t]
        current_year = current_date.year
        
        # Load signals for current year if not cached
        if current_year not in yearly_signals_cache:
            yearly_signals_cache[current_year] = load_yearly_signals(
                current_year, buys_path_template, sells_path_template
            )
        
        buysell_permnos = yearly_signals_cache[current_year]
        
        # Get analyst recommendations for current date
        analyst_permnos = set()
        if analyst_df is not None and len(analyst_df) > 0:
            analyst_permnos = get_signal_permnos_for_date(analyst_df, current_date)
        
        # UNION of buy/sell signals and analyst recommendations
        allowed_permnos = buysell_permnos.intersection(analyst_permnos)
        if len(allowed_permnos) <= 1:
            allowed_permnos = buysell_permnos.union(analyst_permnos)
        
        # Get OOS returns FIRST before any early exits
        oos_data = df[(df['datadate'] == current_date) & (df['permno'].isin(allowed_permnos))]
        oos_returns_series = oos_data.set_index('permno')['ret_fwd_1']
        oos_returns_series = oos_returns_series.dropna()
        oos_returns_dict = oos_returns_series.to_dict()
        
        # Handle early exit cases
        if len(allowed_permnos) == 0:
            if verbose:
                print(f"\n[{t - test_start_idx + 1}/{test_end_idx - test_start_idx + 1}] "
                      f"Date: {current_date.strftime('%Y-%m-%d')}")
                print(f"  ⚠ No signals for {current_date.strftime('%Y-%m-%d')}, recording zero return")
            
            # Process exit for all three portfolios
            for ptype in ['gmv', 'mv', 'msr']:
                turnover, tc, net_return = calculate_exit_transaction_cost(
                    prev_state[ptype]['weights_dict'], 
                    prev_state[ptype]['oos_returns_dict'], 
                    prev_state[ptype]['gross_return'], 
                    transaction_cost,
                    verbose=False
                )
                
                portfolio_results[ptype]['returns'].append(net_return)
                portfolio_results[ptype]['dates'].append(current_date)
                portfolio_results[ptype]['weights'].append({})
                portfolio_results[ptype]['turnover'].append(turnover)
                portfolio_results[ptype]['gross_returns'].append(0.0)
                
                prev_state[ptype] = {'weights_dict': {}, 'oos_returns_dict': {}, 'gross_return': 0.0}
            continue
        
        # Define the lookback window
        window_start_date = all_dates[t - lookback_window]
        window_end_date = all_dates[t - 1]
        
        # Get training data for this window
        train_data = df[(df['datadate'] >= window_start_date) & 
                        (df['datadate'] <= window_end_date) &
                        (df['permno'].isin(allowed_permnos))]
        
        # Pivot to get returns matrix
        returns_pivot = train_data.pivot(index='datadate', columns='permno', values='ret_fwd_1')
        window_dates = all_dates[t - lookback_window : t]
        returns_pivot = returns_pivot.reindex(index=window_dates)

        # Align factors with return realization dates
        factor_dates = [(d + pd.DateOffset(months=1) + pd.offsets.MonthEnd(0)) for d in window_dates]
        
        try:
            factors_window = factors_df.loc[factor_dates]
        except KeyError as e:
            raise ValueError(f"Factor dates not found in factors file. Missing dates: {e}")
        
        if factors_window.isna().any().any():
            if verbose:
                print(f"\n[{t - test_start_idx + 1}/{test_end_idx - test_start_idx + 1}] "
                      f"Date: {current_date.strftime('%Y-%m-%d')}")
                print(f"  ⚠ Missing factor data in window, skipping period")
            continue
        
        # Filter assets with any NaNs
        nan_assets = returns_pivot.columns[returns_pivot.isna().any()]
        filtered_pivot = returns_pivot.drop(columns=nan_assets)
        
        current_assets = filtered_pivot.columns.tolist()
        Y = filtered_pivot.values
        factors = factors_window.values
        n_train, p_current = Y.shape
    
        if verbose:
            print(f"\n[{t - test_start_idx + 1}/{test_end_idx - test_start_idx + 1}] "
                  f"Date: {current_date.strftime('%Y-%m-%d')} | Year: {current_year}")
            print(f"  Window: {window_start_date.strftime('%Y-%m-%d')} to "
                  f"{window_end_date.strftime('%Y-%m-%d')}")
            print(f" Analyst + LLM: {len(allowed_permnos)} | "
                  f" | Assets w/ data: {p_current}")
    
        # Check for valid data
        if n_train < lookback_window or p_current < 2:
            if verbose:
                print(f"  ⚠ Insufficient data (n={n_train}, p={p_current}), recording zero return")
            
            for ptype in ['gmv', 'mv', 'msr']:
                turnover, tc, net_return = calculate_exit_transaction_cost(
                    prev_state[ptype]['weights_dict'], 
                    prev_state[ptype]['oos_returns_dict'], 
                    prev_state[ptype]['gross_return'], 
                    transaction_cost,
                    verbose=False
                )
                
                portfolio_results[ptype]['returns'].append(net_return)
                portfolio_results[ptype]['dates'].append(current_date)
                portfolio_results[ptype]['weights'].append({})
                portfolio_results[ptype]['turnover'].append(turnover)
                portfolio_results[ptype]['gross_returns'].append(0.0)
                
                prev_state[ptype] = {'weights_dict': {}, 'oos_returns_dict': {}, 'gross_return': 0.0}
            continue
        
        try:
            # Demean the returns
            Y_bar = Y.mean(axis=0)
            Y_star = Y - Y_bar
            
            if verbose:
                print(f"  Running Residual Nodewise Regression...")
            Theta_hat = est_ndwcov_factor(Y, factors, ic=ic, lambda_min=True)
            
            if verbose:
                print(f"  Computing portfolio weights...")

            mu = Y.mean(axis=0)
            
            # Compute weights for all three portfolios
            weights_dict = {
                'gmv': gmv_weights(Theta_hat),
                'mv': mv_weights(Theta_hat, mu, target_return=mv_target_return),
                'msr': msr_weights(Theta_hat, mu)
            }
            
            # Create weights dictionaries for each portfolio
            new_weights_dicts = {
                ptype: {asset: weights_dict[ptype][i] for i, asset in enumerate(current_assets)}
                for ptype in ['gmv', 'mv', 'msr']
            }
            
        except Exception as e:
            if verbose:
                print(f"  ✗ Error: {e}")
                print(f"  Recording zero return for all portfolios")
            
            for ptype in ['gmv', 'mv', 'msr']:
                turnover, tc, net_return = calculate_exit_transaction_cost(
                    prev_state[ptype]['weights_dict'], 
                    prev_state[ptype]['oos_returns_dict'], 
                    prev_state[ptype]['gross_return'], 
                    transaction_cost,
                    verbose=False
                )
                
                portfolio_results[ptype]['returns'].append(net_return)
                portfolio_results[ptype]['dates'].append(current_date)
                portfolio_results[ptype]['weights'].append({})
                portfolio_results[ptype]['turnover'].append(turnover)
                portfolio_results[ptype]['gross_returns'].append(0.0)
                
                prev_state[ptype] = {'weights_dict': {}, 'oos_returns_dict': {}, 'gross_return': 0.0}
            continue

        # Process each portfolio type
        for ptype in ['gmv', 'mv', 'msr']:
            new_weights_dict = new_weights_dicts[ptype]
            
            # Normalize weights to sum to 1
            weight_sum = sum(new_weights_dict.values())
            if weight_sum > 1e-10:
                new_weights_dict = {k: v/weight_sum for k, v in new_weights_dict.items()}
            else:
                if verbose and ptype == 'gmv':  # Only print once
                    print(f"  ⚠ Zero weight sum for {ptype.upper()}, recording zero return")
                
                turnover, tc, net_return = calculate_exit_transaction_cost(
                    prev_state[ptype]['weights_dict'], 
                    prev_state[ptype]['oos_returns_dict'], 
                    prev_state[ptype]['gross_return'], 
                    transaction_cost,
                    verbose=False
                )
                
                portfolio_results[ptype]['returns'].append(net_return)
                portfolio_results[ptype]['dates'].append(current_date)
                portfolio_results[ptype]['weights'].append({})
                portfolio_results[ptype]['turnover'].append(turnover)
                portfolio_results[ptype]['gross_returns'].append(0.0)
                
                prev_state[ptype] = {'weights_dict': {}, 'oos_returns_dict': {}, 'gross_return': 0.0}
                continue
            
            # Find common assets between weights and returns
            common_assets = set(new_weights_dict.keys()) & set(oos_returns_dict.keys())
            
            if len(common_assets) == 0:
                if verbose and ptype == 'gmv':
                    print(f"  ⚠ No common assets with valid returns")
                
                turnover, tc, net_return = calculate_exit_transaction_cost(
                    prev_state[ptype]['weights_dict'], 
                    prev_state[ptype]['oos_returns_dict'], 
                    prev_state[ptype]['gross_return'], 
                    transaction_cost,
                    verbose=False
                )
                
                portfolio_results[ptype]['returns'].append(net_return)
                portfolio_results[ptype]['dates'].append(current_date)
                portfolio_results[ptype]['weights'].append({})
                portfolio_results[ptype]['turnover'].append(turnover)
                portfolio_results[ptype]['gross_returns'].append(0.0)
                
                prev_state[ptype] = {'weights_dict': {}, 'oos_returns_dict': {}, 'gross_return': 0.0}
                continue
            
            # Filter to common assets and renormalize
            common_weights = {a: new_weights_dict[a] for a in common_assets}
            common_weight_sum = sum(common_weights.values())
            if common_weight_sum > 1e-10:
                common_weights = {k: v/common_weight_sum for k, v in common_weights.items()}
            else:
                turnover, tc, net_return = calculate_exit_transaction_cost(
                    prev_state[ptype]['weights_dict'], 
                    prev_state[ptype]['oos_returns_dict'], 
                    prev_state[ptype]['gross_return'], 
                    transaction_cost,
                    verbose=False
                )
                
                portfolio_results[ptype]['returns'].append(net_return)
                portfolio_results[ptype]['dates'].append(current_date)
                portfolio_results[ptype]['weights'].append({})
                portfolio_results[ptype]['turnover'].append(turnover)
                portfolio_results[ptype]['gross_returns'].append(0.0)
                
                prev_state[ptype] = {'weights_dict': {}, 'oos_returns_dict': {}, 'gross_return': 0.0}
                continue
            
            # Compute gross portfolio return
            gross_return = sum(common_weights[a] * oos_returns_dict[a] for a in common_assets)
            
            # Sanity check
            if np.isnan(gross_return) or np.isinf(gross_return):
                turnover, tc, net_return = calculate_exit_transaction_cost(
                    prev_state[ptype]['weights_dict'], 
                    prev_state[ptype]['oos_returns_dict'], 
                    prev_state[ptype]['gross_return'], 
                    transaction_cost,
                    verbose=False
                )
                
                portfolio_results[ptype]['returns'].append(net_return)
                portfolio_results[ptype]['dates'].append(current_date)
                portfolio_results[ptype]['weights'].append({})
                portfolio_results[ptype]['turnover'].append(turnover)
                portfolio_results[ptype]['gross_returns'].append(0.0)
                
                prev_state[ptype] = {'weights_dict': {}, 'oos_returns_dict': {}, 'gross_return': 0.0}
                continue
            
            # Calculate transaction costs
            if len(prev_state[ptype]['weights_dict']) > 0:
                # Adjust previous weights for returns
                adjusted_prev = {}
                prev_weights_dict = prev_state[ptype]['weights_dict']
                prev_oos_returns_dict = prev_state[ptype]['oos_returns_dict']
                prev_gross_return = prev_state[ptype]['gross_return']
                
                for asset, prev_w in prev_weights_dict.items():
                    if asset in prev_oos_returns_dict:
                        prev_r = prev_oos_returns_dict[asset]
                        if abs(1 + prev_gross_return) > 1e-6:
                            adjusted_prev[asset] = prev_w * (1 + prev_r) / (1 + prev_gross_return)
                        else:
                            adjusted_prev[asset] = 0.0
                    else:
                        if abs(1 + prev_gross_return) > 1e-6:
                            adjusted_prev[asset] = prev_w / (1 + prev_gross_return)
                        else:
                            adjusted_prev[asset] = 0.0
                
                # Calculate turnover
                all_assets = set(adjusted_prev.keys()) | set(common_weights.keys())
                turnover = 0.0
                for asset in all_assets:
                    old_w = adjusted_prev.get(asset, 0.0)
                    new_w = common_weights.get(asset, 0.0)
                    turnover += abs(new_w - old_w)
                
                tc = transaction_cost * (1 + gross_return) * turnover
            else:
                # First period
                turnover = sum(abs(w) for w in common_weights.values())
                tc = transaction_cost * (1 + gross_return) * turnover
            
            # Net return
            net_return = gross_return - tc
            
            # Store results
            portfolio_results[ptype]['returns'].append(net_return)
            portfolio_results[ptype]['dates'].append(current_date)
            portfolio_results[ptype]['weights'].append(common_weights.copy())
            portfolio_results[ptype]['turnover'].append(turnover)
            portfolio_results[ptype]['gross_returns'].append(gross_return)
            
            # Update previous state
            prev_state[ptype] = {
                'weights_dict': common_weights.copy(),
                'oos_returns_dict': {a: oos_returns_dict[a] for a in common_assets},
                'gross_return': gross_return
            }
        
        if verbose:
            print(f"  GMV  - Gross: {portfolio_results['gmv']['gross_returns'][-1]:>8.5f} | "
                  f"Turnover: {portfolio_results['gmv']['turnover'][-1]:>6.4f} | "
                  f"Net: {portfolio_results['gmv']['returns'][-1]:>8.5f}")
            print(f"  MV   - Gross: {portfolio_results['mv']['gross_returns'][-1]:>8.5f} | "
                  f"Turnover: {portfolio_results['mv']['turnover'][-1]:>6.4f} | "
                  f"Net: {portfolio_results['mv']['returns'][-1]:>8.5f}")
            print(f"  MSR  - Gross: {portfolio_results['msr']['gross_returns'][-1]:>8.5f} | "
                  f"Turnover: {portfolio_results['msr']['turnover'][-1]:>6.4f} | "
                  f"Net: {portfolio_results['msr']['returns'][-1]:>8.5f}")

    if verbose:
        print("\n" + "="*60)
        print("BACKTEST COMPLETE")
        print("="*60)
    
    # --- 4. Compile Results for Each Portfolio ---
    results_dict = {}
    
    for ptype in ['gmv', 'mv', 'msr']:
        results_df = pd.DataFrame({
            'date': portfolio_results[ptype]['dates'],
            'portfolio_return': portfolio_results[ptype]['returns'],
            'portfolio_gross_return': portfolio_results[ptype]['gross_returns'],
            'portfolio_weights': portfolio_results[ptype]['weights'],
            'portfolio_turnover': portfolio_results[ptype]['turnover']
        })
        results_df['cumulative_return'] = (1 + results_df['portfolio_return']).cumprod() - 1
        
        # Compute overall metrics
        if len(portfolio_results[ptype]['returns']) > 0:
            returns = portfolio_results[ptype]['returns']
            mean_return = np.mean(returns)
            variance = np.var(returns, ddof=1)
            sharpe_ratio = mean_return / np.sqrt(variance) if variance > 0 else 0
            
            # Annualized metrics (monthly data)
            annual_return = mean_return * 12
            annual_volatility = np.sqrt(variance * 12)
            annual_sharpe = annual_return / annual_volatility if annual_volatility > 0 else 0
            
            metrics = {
                'mean_return': mean_return,
                'variance': variance,
                'sharpe_ratio': sharpe_ratio,
                'annual_return': annual_return,
                'annual_volatility': annual_volatility,
                'annual_sharpe_ratio': annual_sharpe,
                'total_return': results_df['cumulative_return'].iloc[-1],
                'avg_turnover': np.mean(portfolio_results[ptype]['turnover']),
                'n_periods': len(returns),
                'n_zero_periods': sum(1 for r in returns if r == 0)
            }
        else:
            metrics = {
                'mean_return': 0,
                'variance': 0,
                'sharpe_ratio': 0,
                'annual_return': 0,
                'annual_volatility': 0,
                'annual_sharpe_ratio': 0,
                'total_return': 0,
                'avg_turnover': 0,
                'n_periods': 0,
                'n_zero_periods': 0
            }
        
        results_dict[ptype] = (results_df, metrics)
    
    return results_dict

In [3]:
df = pd.read_csv('../../green cleaned.csv', dtype={'ncusip': 'string'})
df['ret_fwd_1'] = df.groupby('permno')['ret_excess'].shift(-1)

results = backtest_nodewise_gmv_yearly(
    df=df,
    factors_path='../AI Portfolio Selection/factors_ff_monthly_raw.csv',
    ic='GIC',  # or 'BIC', 'WIC', 'AIC', 'cv'
    test_start_date='2015-01-31',
    test_end_date='2024-04-30',
    lookback_window=180,
    transaction_cost=0.001,
    buys_path_template='../AI Portfolio Selection/buys_{}.csv',
    sells_path_template='../AI Portfolio Selection/sells_{}.csv',
    analyst_rec_path='monthly_mean_recommendations_decay.csv',
    verbose=True
)

# Access individual results
gmv_df, gmv_metrics = results['gmv']
mv_df, mv_metrics = results['mv']
msr_df, msr_metrics = results['msr']

# Compare Sharpe ratios
print(f"GMV Sharpe: {gmv_metrics['annual_sharpe_ratio']:.3f}")
print(f"MV Sharpe: {mv_metrics['annual_sharpe_ratio']:.3f}")
print(f"MSR Sharpe: {msr_metrics['annual_sharpe_ratio']:.3f}")

Loaded analyst recommendations: 37527 records
STARTING BACKTEST WITH YEARLY SIGNALS + ANALYST RECS

[1/112] Date: 2015-01-31 | Year: 2015
  Window: 2000-01-31 to 2014-12-31
 Analyst + LLM: 5 |  | Assets w/ data: 3
  Running Residual Nodewise Regression...
  Computing portfolio weights...
  GMV  - Gross:  0.02188 | Turnover: 1.0000 | Net:  0.02086
  MV   - Gross:  0.07175 | Turnover: 1.3194 | Net:  0.07033
  MSR  - Gross:  0.05548 | Turnover: 1.0300 | Net:  0.05439

[2/112] Date: 2015-02-28 | Year: 2015
  Window: 2000-02-29 to 2015-01-31
 Analyst + LLM: 6 |  | Assets w/ data: 3
  Running Residual Nodewise Regression...
  Computing portfolio weights...
  GMV  - Gross: -0.06860 | Turnover: 0.9736 | Net: -0.06951
  MV   - Gross: -0.06733 | Turnover: 0.3834 | Net: -0.06768
  MSR  - Gross: -0.06813 | Turnover: 0.2863 | Net: -0.06840

[3/112] Date: 2015-03-31 | Year: 2015
  Window: 2000-03-31 to 2015-02-28
 Analyst + LLM: 8 |  | Assets w/ data: 4
  Running Residual Nodewise Regression...
  Co

In [4]:
print(f"\n GMV")
print(f"Annualized Sharpe Ratio: {gmv_metrics['annual_sharpe_ratio']:.4f}")
print(f"Mean Return: {gmv_metrics['mean_return']*12:.4f}")
print(f"Variance: {gmv_metrics['variance']*12:.4f}")
print(f"Avg Turnover: {gmv_metrics['avg_turnover']:.4f}")

print(f"\n MV")
print(f"Annualized Sharpe Ratio: {mv_metrics['annual_sharpe_ratio']:.4f}")
print(f"Mean Return: {mv_metrics['mean_return']*12:.4f}")
print(f"Variance: {mv_metrics['variance']*12:.4f}")
print(f"Avg Turnover: {mv_metrics['avg_turnover']:.4f}")

print(f"\n MSR")
print(f"Annualized Sharpe Ratio: {msr_metrics['annual_sharpe_ratio']:.4f}")
print(f"Mean Return: {msr_metrics['mean_return']*12:.4f}")
print(f"Variance: {msr_metrics['variance']*12:.4f}")
print(f"Avg Turnover: {msr_metrics['avg_turnover']:.4f}")


 GMV
Annualized Sharpe Ratio: 0.4506
Mean Return: 0.0735
Variance: 0.0266
Avg Turnover: 1.9216

 MV
Annualized Sharpe Ratio: 0.2283
Mean Return: 0.0419
Variance: 0.0337
Avg Turnover: 2.3491

 MSR
Annualized Sharpe Ratio: 0.1093
Mean Return: 0.0224
Variance: 0.0419
Avg Turnover: 2.6469
